In [27]:
import os
os.getcwd()

'C:\\Users\\wjdal\\OneDrive\\바탕 화면\\Prediction_Solar_generation\\서부한전_DATA_EDA_1'

In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

In [3]:
power_data = pd.read_csv('./DATA/한국서부발전(주)_태양광 발전 현황_20200927 (1).csv', encoding='euc-kr')
power_data.head(1)

,일자,발전기명,01시,02시,03시,04시,05시,06시,07시,08시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,2017-01-01,영암에프원태양광b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7344000.0,5155200.0,2131200.0,134400.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
power_data_영암 = power_data[power_data['발전기명']=='영암에프원태양광b']
power_data_세종 = power_data[power_data['발전기명']=='세종시폐기물매립장태양광']
power_data_안산 = power_data[power_data['발전기명']=='안산연성정수장태양광']

In [5]:
# Transform Data
def transform_data(pd):
    power_time = list(pd.columns)[2:]
    change_power_data = []
    for i in range(pd.shape[0]):
        target=[]
        for j in range(26):
            if j ==0:
                date = pd.iloc[i,j]
            elif (j!=0) & (j!=1):
                target.append([date+" "+power_time[j-2],pd.iloc[i,j]])
        change_power_data.append(target)

    change_power_data_date = []
    change_power_data_generation = []
    for i in change_power_data:
        for j in i:
            change_power_data_date.append(j[0])
            change_power_data_generation.append(j[1])
    return change_power_data_date, change_power_data_generation


In [6]:
transform_영암= transform_data(power_data_영암)
transform_세종= transform_data(power_data_세종)
transform_안산= transform_data(power_data_안산)

In [7]:
# Extract data by year
def extract_data_2017(transform_data):
    total_power_generation = pd.DataFrame(data = transform_data[1], index=transform_data[0], columns=['generation'])
    total_power_generation_2017 = total_power_generation.iloc[:8760,:]
    return total_power_generation_2017

def extract_data_2018(transform_data):
    total_power_generation = pd.DataFrame(data = transform_data[1], index=transform_data[0], columns=['generation'])
    total_power_generation_2018 = total_power_generation.iloc[8760:17520,:]
    return total_power_generation_2018

def extract_data_2019(transform_data):
    total_power_generation = pd.DataFrame(data = transform_data[1], index=transform_data[0], columns=['generation'])
    total_power_generation_2019= total_power_generation.iloc[17520:26280,:]
    return total_power_generation_2019

def extract_data_2020(transform_data):
    total_power_generation = pd.DataFrame(data = transform_data[1], index=transform_data[0], columns=['generation'])
    total_power_generation_2020 = total_power_generation.iloc[26280:,:]
    return total_power_generation_2019

In [8]:
# Categorized by three time zones => (09:00~09:59), (13:00~13:59), (17:00~17:59)
def categorize_time(tranform_data):
    target_datetime_generation=[]
    for i,j in zip(tranform_data[0], tranform_data[1]):
        date_time= i.split(" ")
        if (date_time[1] =='09시') | (date_time[1] =='13시') | (date_time[1] =='17시'):
            target_datetime_generation.append([i,j])
            
    target_datetime_power_generation = pd.DataFrame(data=target_datetime_generation, columns=['date_time','generation'] )
    
    return target_datetime_power_generation

In [9]:
categorize_영암 = categorize_time(transform_영암)
categorize_세종 = categorize_time(transform_세종)
categorize_안산 = categorize_time(transform_안산)

##### 회귀모델을 위한 데이터 전처리 ( 세개의 시간대로만 분류)

In [11]:
def divid_by_hour(categorize_data, spec_hours):
    time_09h=[]
    time_13h=[]
    time_17h=[]
    cnt=0
    for i in categorize_data['date_time']:
        date_time = i.split(" ")
        if date_time[1]=='09시':
            time_09h.append([i,categorize_data['generation'][cnt]])
        elif date_time[1]=='13시':
            time_13h.append([i,categorize_data['generation'][cnt]])
        elif date_time[1]=='17시':
            time_17h.append([i,categorize_data['generation'][cnt]])
        cnt+=1
    
    if spec_hours=='09시':
        power_09h = pd.DataFrame(data=time_09h, columns=['date','generation'])
        return power_09h
    elif spec_hours=='13시':
        power_13h = pd.DataFrame(data=time_13h, columns=['date','generation'])
        return power_13h
    elif spec_hours=='17시':
        power_17h = pd.DataFrame(data=time_17h, columns=['date','generation'])
        return power_17h

In [13]:
total_09h_안산=divid_by_hour(categorize_안산,'09시')
total_13h_안산=divid_by_hour(categorize_안산,'13시')
total_17h_안산=divid_by_hour(categorize_안산,'17시')

In [18]:
total_09h_세종=divid_by_hour(categorize_세종,'09시')
total_13h_세종=divid_by_hour(categorize_세종,'13시')
total_17h_세종=divid_by_hour(categorize_세종,'17시')

In [22]:
total_09h_영암=divid_by_hour(categorize_영암,'09시')
total_13h_영암=divid_by_hour(categorize_영암,'13시')
total_17h_영암=divid_by_hour(categorize_영암,'17시')

###### EDA를 위한 전처리

In [ ]:
def divid_year_2017(categorize_data):
    divid_2017  = categorize_data.iloc[:1095,:]
    return divid_2017

def divid_year_2018(categorize_data):
    divid_2018  = categorize_data.iloc[1095:2190,:]
    return divid_2018

def divid_year_2019(categorize_data):
    divid_2019  = categorize_data.iloc[2190:3285,:]
    return divid_2019

def divid_year_2020(categorize_data):
    divid_2020  = categorize_data.iloc[3285:,:]
    return divid_2020

In [ ]:
divid_2017_영암= divid_year_2017(categorize_영암)
divid_2018_영암= divid_year_2018(categorize_영암)
divid_2019_영암= divid_year_2019(categorize_영암)

In [ ]:
divid_2017_세종= divid_year_2017(categorize_세종)
divid_2018_세종= divid_year_2018(categorize_세종)
divid_2019_세종= divid_year_2019(categorize_세종)

In [ ]:
divid_2017_안산= divid_year_2017(categorize_안산)
divid_2018_안산= divid_year_2018(categorize_안산)
divid_2019_안산= divid_year_2019(categorize_안산)

In [ ]:
generation_in_time = ['09시','10시','11시','12시','13시','14시','15시','16시','17시','18시'] 
def slice_time(tranform_data, generation_in_time):
    target_datetime_generation=[]
    for i,j in zip(tranform_data[0], tranform_data[1]):
        date_time= i.split(" ")
        if date_time[1] in generation_in_time:
            target_datetime_generation.append([i,j])
            
    target_datetime_power_generation = pd.DataFrame(data=target_datetime_generation, columns=['date_time','generation'] )
    
    return target_datetime_power_generation

In [ ]:
slice_영암 = slice_time(transform_영암, generation_in_time)
slice_세종 = slice_time(transform_세종, generation_in_time)
slice_안산 = slice_time(transform_안산, generation_in_time)

In [ ]:
def slice_year_2017(slice_data):
    slice_2017  = slice_data.iloc[:3650,:]
    return slice_2017

def slice_year_2018(slice_data):
    slice_2018  = slice_data.iloc[3650:7300,:]
    return slice_2018

def slice_year_2019(slice_data):
    slice_2019  = slice_data.iloc[7300:10950,:]
    return slice_2019

def slice_year_2020(slice_data):
    slice_2020  = slice_data.iloc[10950:,:]
    return slice_2020

In [ ]:
slice_2017_영암= slice_year_2017(slice_영암)
slice_2017_영암=slice_2017_영암.reset_index()
slice_2017_영암 = slice_2017_영암.drop(['index','date_time'], axis=1)

slice_2018_영암= slice_year_2018(slice_영암)
slice_2018_영암= slice_2018_영암.reset_index()
slice_2018_영암 = slice_2018_영암.drop(['index','date_time'], axis=1)

slice_2019_영암= slice_year_2019(slice_영암)
slice_2019_영암= slice_2019_영암.reset_index()
slice_2019_영암 = slice_2019_영암.drop(['index','date_time'], axis=1)

slice_2020_영암= slice_year_2020(slice_영암)
slice_2020_영암 = slice_2020_영암.reset_index()
slice_2020_영암 = slice_2020_영암.drop(['index','date_time'], axis=1)


In [ ]:
# 3개년 발전량의 일별 시간대별 평균값
generation_mean_영암 = (slice_2017_영암+slice_2018_영암+slice_2019_영암)/3
# generation_mean_영암

In [ ]:
# 3개년의 중앙값으로 발전량 정렬
median=[]
for i, j, k in zip(slice_2017_영암['generation'], slice_2018_영암['generation'], slice_2019_영암['generation']):
    lists =[]
    lists.append(i)
    lists.append(j)
    lists.append(k)
    lists = sorted(lists)
    median.append(lists[1])

generation_median_영암 = pd.DataFrame(data = median, columns=['generation'])
# generation_median_영암

### 기상 변수에 대한 설명
#####  일조(hr)	: 태양광선이 구름이나 안개등에 의해서 차단되지 않고 지표면을 비춘시간, 시의 단위로써 표시하며 10분위까지의 값으로 표현
##### 일사(MJ/m2)(일사량) : 일사의 세기로 태양으로부터 지구로 복사되는 에너지를 일사라 한다. 일사량의 측정은 태양광선의 진행방향에 직각인 1cm2의 넓이에 1분 동안 복사된 양으로 측정한다.
##### 전운량(10분위) : 전운량은 운량과 같은뜻으로 하늘을 덮고 있는 구름의 양의 비율, 하늘 전체(눈에 보이는 범위)의 몇 %쯤이 구름으로 덮여 있는지에 따라 0부터 10까지의 계급으로 나눈다
##### 풍향(16방위) : 풍향은 바람이 불어오는 방향으로 나타내며 보통 16방위를 사용, 16방위는 360도의 원판을 22.5도 간격으로 16등분

#### 세 시간대별, 발전소별 전체 날씨 데이터 정렬

In [28]:
wea_2017 = pd.read_csv('./DATA/서부발전_2017.csv', encoding='euc-kr')
wea_2018 = pd.read_csv('./DATA/서부발전_2018.csv', encoding='euc-kr')
wea_2019 = pd.read_csv('./DATA/서부발전_2019.csv', encoding='euc-kr')
wea_2020 = pd.read_csv('./DATA/서부발전_2020.csv', encoding='euc-kr')

In [57]:
wea_2017_안산=wea_2017[wea_2017['지점명']=='수원'] 
wea_2018_안산=wea_2018[wea_2018['지점명']=='수원'] 
wea_2019_안산=wea_2019[wea_2019['지점명']=='수원']
wea_2020_안산=wea_2020[wea_2020['지점명']=='수원'] 

In [58]:
wea_2017_영암=wea_2017[wea_2017['지점명']=='목포'] 
wea_2018_영암=wea_2018[wea_2018['지점명']=='목포'] 
wea_2019_영암=wea_2019[wea_2019['지점명']=='목포']
wea_2020_영암=wea_2020[wea_2020['지점명']=='목포'] 

In [59]:
wea_2017_세종=wea_2017[wea_2017['지점명']=='대전'] 
wea_2018_세종=wea_2018[wea_2018['지점명']=='대전'] 
wea_2019_세종=wea_2019[wea_2019['지점명']=='대전']
wea_2020_세종=wea_2020[wea_2020['지점명']=='대전'] 

In [60]:
frame_안산=[wea_2017_안산,wea_2018_안산,wea_2019_안산,wea_2020_안산]
total_weather_안산 = pd.concat(frame)

In [63]:
frame_영암=[wea_2017_영암,wea_2018_영암,wea_2019_영암,wea_2020_영암]
total_weather_영암 = pd.concat(frame)

In [62]:
frame_세종=[wea_2017_세종,wea_2018_세종,wea_2019_세종,wea_2020_세종]
total_weather_세종 = pd.concat(frame)

In [51]:
def dividup_by_hour(concat_data, spec_hours):
    time_09h=[]
    time_13h=[]
    time_17h=[]
    cnt=0
    for i in concat_data['일시']:
        date_time = i.split(" ")
        if date_time[1]=='09:00':
            time_09h.append(list(concat_data.iloc[cnt,3:]))
        elif date_time[1]=='13:00':
            time_13h.append(list(concat_data.iloc[cnt,3:]))
        elif date_time[1]=='17:00':
            time_17h.append(list(concat_data.iloc[cnt,3:]))
        cnt+=1
    
    if spec_hours=='09시':
        power_09h = pd.DataFrame(data=time_09h, columns=['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전운량(10분위)', '운형(운형약어)', '지면온도(°C)'])
        return power_09h
    elif spec_hours=='13시':
        power_13h = pd.DataFrame(data=time_13h, columns=['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전운량(10분위)', '운형(운형약어)', '지면온도(°C)'])
        return power_13h
    elif spec_hours=='17시':
        power_17h = pd.DataFrame(data=time_17h, columns=['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전운량(10분위)', '운형(운형약어)', '지면온도(°C)'])
        return power_17h


In [64]:
total_weather_data_09h_안산= dividup_by_hour(total_weather_안산, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)
total_weather_data_13h_안산= dividup_by_hour(total_weather_안산, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)
total_weather_data_17h_안산= dividup_by_hour(total_weather_안산, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)

In [65]:
total_weather_data_09h_세종= dividup_by_hour(total_weather_세종, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)
total_weather_data_13h_세종= dividup_by_hour(total_weather_세종, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)
total_weather_data_17h_세종= dividup_by_hour(total_weather_세종, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)

In [66]:
total_weather_data_09h_영암= dividup_by_hour(total_weather_영암, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)
total_weather_data_13h_영암= dividup_by_hour(total_weather_영암, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)
total_weather_data_17h_영암= dividup_by_hour(total_weather_영암, '09시').drop(['운형(운형약어)','풍향(16방위)'],axis=1).fillna(0)

#### 날씨 데이터 + 발전량 데이터

In [67]:
total_weather_data_09h_안산

,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전운량(10분위),지면온도(°C)
0,-0.2,0.0,0.3,100.0,0.0,0.09,9.0,-0.4
1,4.1,0.6,1.1,97.0,0.0,0.03,9.0,0.7
2,-0.1,0.0,0.5,86.0,0.6,0.19,0.0,-0.4
3,1.7,0.0,0.1,100.0,0.0,0.15,0.0,-0.1
4,1.6,0.0,0.2,77.0,0.0,0.12,8.0,-0.2
...,...,...,...,...,...,...,...,...
1272,20.3,0.0,2.3,97.0,0.0,0.23,10.0,21.4
1273,23.2,0.0,1.0,74.0,0.0,0.88,10.0,25.5
1274,23.6,0.0,2.2,85.0,0.5,1.08,7.0,26.8
1275,24.6,0.0,2.1,74.0,0.5,1.28,9.0,27.3


In [68]:
total_09h_안산

,date,generation
0,2017-01-01 09시,22800.0
1,2017-01-02 09시,8400.0
2,2017-01-03 09시,34800.0
3,2017-01-04 09시,32400.0
4,2017-01-05 09시,24000.0
...,...,...
1272,2020-06-26 09시,44400.0
1273,2020-06-27 09시,214800.0
1274,2020-06-28 09시,488400.0
1275,2020-06-29 09시,549600.0


In [74]:
total_weather_data_09h_안산['발전량'] = total_09h_안산['generation']
total_09h_power_weather_안산 = total_weather_data_09h_안산

total_weather_data_13h_안산['발전량'] = total_13h_안산['generation']
total_13h_power_weather_안산 = total_weather_data_13h_안산

total_weather_data_17h_안산['발전량'] = total_17h_안산['generation']
total_17h_power_weather_안산 = total_weather_data_17h_안산

In [75]:
total_weather_data_09h_세종['발전량'] = total_09h_세종['generation']
total_09h_power_weather_세종 = total_weather_data_09h_세종

total_weather_data_13h_세종['발전량'] = total_13h_세종['generation']
total_13h_power_weather_세종 = total_weather_data_13h_세종

total_weather_data_17h_세종['발전량'] = total_17h_세종['generation']
total_17h_power_weather_세종 = total_weather_data_17h_세종

In [76]:
total_weather_data_09h_영암['발전량'] = total_09h_영암['generation']
total_09h_power_weather_영암 = total_weather_data_09h_영암

total_weather_data_13h_영암['발전량'] = total_13h_영암['generation']
total_13h_power_weather_영암 = total_weather_data_13h_영암

total_weather_data_17h_영암['발전량'] = total_17h_영암['generation']
total_17h_power_weather_영암 = total_weather_data_17h_영암

In [91]:
total_09h_power_weather_안산.to_csv('total_09h_power_weather_안산.csv')
total_13h_power_weather_안산.to_csv('total_13h_power_weather_안산.csv')
total_17h_power_weather_안산.to_csv('total_17h_power_weather_안산.csv')

In [ ]:
total_09h_power_weather_세종.to_csv('total_09h_power_weather_세종.csv')
total_13h_power_weather_세종.to_csv('total_13h_power_weather_세종.csv')
total_17h_power_weather_세종.to_csv('total_17h_power_weather_세종.csv')

In [ ]:
total_09h_power_weather_영암.to_csv('total_09h_power_weather_영암.csv')
total_13h_power_weather_영암.to_csv('total_13h_power_weather_영암.csv')
total_17h_power_weather_영암.to_csv('total_17h_power_weather_영암.csv')

---------------------------------------------------------------------------------------

In [29]:
# Categorized as a specific area in a particular year

def classification_area_year(weather_data, area_name):
    if area_name=='영암':
        weather= weather_data[weather_data['지점명']=='목포'].iloc[:,2:]
        return weather
    elif area_name=='세종':
        weather= weather_data[weather_data['지점명']=='대전'].iloc[:,2:]
        return weather
    elif area_name=='안산':
        weather= weather_data[weather_data['지점명']=='수원'].iloc[:,2:]
        return weather

In [31]:
wea_2017_영암= classification_area_year(wea_2017, '영암')
wea_2018_영암= classification_area_year(wea_2018, '영암')
wea_2019_영암= classification_area_year(wea_2019, '영암')

In [ ]:
wea_2017_세종= classification_area_year(wea_2017, '세종')
wea_2018_세종= classification_area_year(wea_2018, '세종')
wea_2019_세종= classification_area_year(wea_2019, '세종')

In [ ]:
wea_2017_안산= classification_area_year(wea_2017, '안산')
wea_2018_안산= classification_area_year(wea_2018, '안산')
wea_2019_안산= classification_area_year(wea_2019, '안산')

In [ ]:
# Categorized by three time zones => (09:00~09:59), (13:00~13:59), (17:00~17:59)
# and conbine power data, weather data
def conbine_power_weather(specific_weather, divid_data):
    target_datetime_weather=[]
    cnt=1
    for i in specific_weather['일시']:
        date_time= i.split(" ")
        if (date_time[1] =='09:00') | (date_time[1] =='13:00') | (date_time[1] =='17:00'):
            data= list(specific_weather.iloc[cnt,1:])
            target_datetime_weather.append(data)
        cnt+=1
        
    conbined =[]
    for i, j in zip(target_datetime_weather, list(divid_data['generation'])):
        i.append(j)
        conbined.append(i)
        
    power_weather = pd.DataFrame(data=conbined, index=divid_data['date_time'], columns=['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전운량(10분위)', '운형(운형약어)', '지면온도(°C)', '발전량'])

    return power_weather

In [ ]:
power_weather_2017_영암 = conbine_power_weather(wea_2017_영암, divid_2017_영암)
power_weather_2018_영암 = conbine_power_weather(wea_2018_영암, divid_2018_영암)
power_weather_2019_영암 = conbine_power_weather(wea_2019_영암, divid_2019_영암)

In [ ]:
power_weather_2017_세종 = conbine_power_weather(wea_2017_세종, divid_2017_세종)
power_weather_2018_세종 = conbine_power_weather(wea_2018_세종, divid_2018_세종)
power_weather_2019_세종 = conbine_power_weather(wea_2019_세종, divid_2019_세종)

In [ ]:
power_weather_2017_안산 = conbine_power_weather(wea_2017_안산, divid_2017_안산)
power_weather_2018_안산 = conbine_power_weather(wea_2018_안산, divid_2018_안산)
power_weather_2019_안산 = conbine_power_weather(wea_2019_안산, divid_2019_안산)

In [ ]:
# # 기온
# fig, ax = plt.subplots(figsize=(8,5))

# ax.plot(power_weather_2017_영암.index, power_weather_2017_영암['기온(°C)'], color='b')
# ax.set_title('2017 Weather : temperature')

In [ ]:
# # 일사
# fig, ax = plt.subplots(figsize=(8,5))

# ax.plot(power_weather_2017_영암.index, power_weather_2017_영암['일사(MJ/m2)'], color='b')
# ax.set_title('2017 Weather : Solar Radiation')

In [ ]:
# # 전운량
# fig, ax = plt.subplots(figsize=(8,5))

# ax.plot(power_weather_2017_영암.index, power_weather_2017_영암['전운량(10분위)'], color='b')
# ax.set_title('2017 Weather : Clouds')

In [ ]:
# sns.pairplot(power_weather_2017_영암, x_vars=['일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)'],y_vars=['발전량'])

# sns.pairplot(power_weather_2017_영암, x_vars=['일시', '일조(hr)', '일사(MJ/m2)', '전운량(10분위)', '지면온도(°C)'] ,y_vars=['발전량'])


In [ ]:
# # 태양광 발전과 일사량사이의 상관관계
# sns.jointplot(x='일사(MJ/m2)', y='발전량', data=power_weather_2017_영암, kind='reg')

In [ ]:
# 태양광 발전과 습도사이의 상관관계
# sns.jointplot(x='습도(%)', y='발전량', data=power_weather_2017_영암, kind='reg')

In [ ]:
# Distinguish between three hours
# parameter : Amount of power generation and weather information in a specific area in a specific year 
# ex) power_weather_2017_영암
def distinguish_hours(spec_data, spec_hours):
    power_weather_09h_list =[]
    power_weather_13h_list =[]
    power_weather_17h_list =[]

    date_lists=list(spec_data.index)
    date_09h=[]
    date_13h=[]
    date_17h=[]
    for i in range(spec_data.shape[0]):
        if i%3==0:
            power_weather_09h_list.append(list(spec_data.iloc[i,:]))
            date_09h.append(date_lists[i])
        elif i%3==1:
            power_weather_13h_list.append(list(spec_data.iloc[i,:]))
            date_13h.append(date_lists[i])
        elif i%3==2:
            power_weather_17h_list.append(list(spec_data.iloc[i,:])) 
            date_17h.append(date_lists[i])
            
    if spec_hours =='9시':
        power_weather_09h = pd.DataFrame(power_weather_09h_list,index=date_09h, columns=['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전운량(10분위)', '운형(운형약어)', '지면온도(°C)', '발전량'])
        return power_weather_09h
    elif spec_hours =='13시':
        power_weather_13h = pd.DataFrame(power_weather_13h_list,index=date_13h, columns=['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전운량(10분위)', '운형(운형약어)', '지면온도(°C)', '발전량'])
        return power_weather_13h
    elif spec_hours =='17시':
        power_weather_17h = pd.DataFrame(power_weather_17h_list,index=date_17h, columns=['기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전운량(10분위)', '운형(운형약어)', '지면온도(°C)', '발전량'])
        return power_weather_17h

###### 영암

In [ ]:
power_weather_2017_09h_영암  =distinguish_hours(power_weather_2017_영암, '9시')
power_weather_2017_13h_영암 =distinguish_hours(power_weather_2017_영암, '13시')
power_weather_2017_17h_영암 =distinguish_hours(power_weather_2017_영암, '17시')

In [ ]:
power_weather_2018_09h_영암  =distinguish_hours(power_weather_2018_영암, '9시')
power_weather_2018_13h_영암 =distinguish_hours(power_weather_2018_영암, '13시')
power_weather_2018_17h_영암 =distinguish_hours(power_weather_2018_영암, '17시')

In [ ]:
power_weather_2019_09h_영암  =distinguish_hours(power_weather_2019_영암, '9시')
power_weather_2019_13h_영암 =distinguish_hours(power_weather_2019_영암, '13시')
power_weather_2019_17h_영암 =distinguish_hours(power_weather_2019_영암, '17시')

###### 세종

In [ ]:
power_weather_2017_09h_세종  =distinguish_hours(power_weather_2017_세종, '9시')
power_weather_2017_13h_세종 =distinguish_hours(power_weather_2017_세종, '13시')
power_weather_2017_17h_세종 =distinguish_hours(power_weather_2017_세종, '17시')

In [ ]:
power_weather_2018_09h_세종  =distinguish_hours(power_weather_2018_세종, '9시')
power_weather_2018_13h_세종 =distinguish_hours(power_weather_2018_세종, '13시')
power_weather_2018_17h_세종 =distinguish_hours(power_weather_2018_세종, '17시')

In [ ]:
power_weather_2019_09h_세종  =distinguish_hours(power_weather_2019_세종, '9시')
power_weather_2019_13h_세종 =distinguish_hours(power_weather_2019_세종, '13시')
power_weather_2019_17h_세종 =distinguish_hours(power_weather_2019_세종, '17시')

###### 안산

In [ ]:
power_weather_2017_09h_안산  =distinguish_hours(power_weather_2017_안산, '9시')
power_weather_2017_13h_안산 =distinguish_hours(power_weather_2017_안산, '13시')
power_weather_2017_17h_안산 =distinguish_hours(power_weather_2017_안산, '17시')

In [ ]:
power_weather_2018_09h_안산  =distinguish_hours(power_weather_2018_안산, '9시')
power_weather_2018_13h_안산 =distinguish_hours(power_weather_2018_안산, '13시')
power_weather_2018_17h_안산 =distinguish_hours(power_weather_2018_안산, '17시')

In [ ]:
power_weather_2019_09h_안산  =distinguish_hours(power_weather_2019_안산, '9시')
power_weather_2019_13h_안산 =distinguish_hours(power_weather_2019_안산, '13시')
power_weather_2019_17h_안산 =distinguish_hours(power_weather_2019_안산, '17시')

In [ ]:
# # 09:00~09:59 까지의 날씨 데이터 EDA

# # 기온
# fig, ax = plt.subplots(figsize=(8,5))

# ax.plot(power_weather_2017_09h_영암.index, power_weather_2017_09h_영암['기온(°C)'], color='b')
# ax.set_title('2017 Weather : temperature')

# # 일사량
# fig, ax = plt.subplots(figsize=(8,5))

# ax.plot(power_weather_2017_09h_영암.index, power_weather_2017_09h_영암['일사(MJ/m2)'], color='b')
# ax.set_title('2017 Weather : Solar Radiaion')

# # 습도
# fig, ax = plt.subplots(figsize=(8,5))

# ax.plot(power_weather_2017_09h_영암.index, power_weather_2017_09h_영암['습도(%)'], color='b')
# ax.set_title('2017 Weather : Humidity')

In [ ]:
# sns.pairplot(power_weather_2017_09h_영암, x_vars=['일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)'],y_vars=['발전량'])

# sns.pairplot(power_weather_2017_09h_영암, x_vars=['일시', '일조(hr)', '일사(MJ/m2)', '전운량(10분위)', '지면온도(°C)'] ,y_vars=['발전량'])


In [ ]:
# 태양광 발전과 일사량사이의 상관관계 (09:00~09:59)
# sns.jointplot(x='일사(MJ/m2)', y='발전량', data=power_weather_2017_09h_영암, kind='reg')

In [ ]:
# 태양광 발전과 습도사이의 상관관계 (09:00~09:59)
# sns.jointplot(x='습도(%)', y='발전량', data=power_weather_2017_09h_영암, kind='reg')

In [ ]:
# 상관계수 행렬
power_weather_2017_09h_영암.corr()

##### 영암 발전소의 (09:00~09:59) 까지의 발전량은 일사량이 가장많은 상관관계(0.875553), 지면온도(0.663508), 일조량(0.654889) 등이 다음으로 양의 상관관계를 보이고 전운량(-0.603045), 습도(-0.401793)가 음의 상관관계를 보임

In [ ]:
power_weather_2017_13h_영암.corr()

#####  영암 발전소의 (13:00~13:59) 까지의 발전량은 일사량이 가장많은 상관관계(0.795109) 일조량(0.772229) 등이 다음으로 양의 상관관계를 보이고 전운량(-0.723233	), 습도(-0.602046)가 음의 상관관계를 보임
##### 9시와는 다르게 기온과 지면온도는 발전량에 미미한 영향을 보여주고 있고 전운량의 음의 상관관계가 강해졌다


In [ ]:
power_weather_2017_17h_영암.corr()

#####  영암 발전소의 (17:00~17:59) 까지의 발전량은 일조량(0.832069)  일사량(0.816160) 등이 다음으로 양의 상관관계를 보이고 전운량(-0.552438	)가 음의 상관관계를 보임
##### 17시에는 일사량보다 일조량이 더많은 양의 상관관계를 보여주고있고 전운량이 여전히 가장 강한 음의 상관관계를 보이고있음


#### 발전량을 예측 하는데 있어서 주의 깊게 봐야할 변수는 일조, 일사, 전운량, 습도, 지면온도 등을 집중적으로 봐야할것 같다

In [ ]:
# power_weather_2017_09h_세종.corr()

In [ ]:
# power_weather_2017_13h_세종.corr()

In [ ]:
# power_weather_2017_17h_세종.corr()

In [ ]:
power_weather_2017_09h_안산.corr()

In [ ]:
power_weather_2017_13h_안산.corr()

In [ ]:
power_weather_2017_17h_안산.corr()

In [ ]:
a=power_weather_2017_09h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
b=power_weather_2018_09h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
c=power_weather_2019_09h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
power_weather_09h_영암_mean = (a+b+c)/3
power_weather_09h_영암_mean.corr()

In [ ]:
a=power_weather_2017_13h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
b=power_weather_2018_13h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
c=power_weather_2019_13h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
power_weather_13h_영암_mean = (a+b+c)/3
power_weather_13h_영암_mean.corr()

In [ ]:
a=power_weather_2017_17h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
b=power_weather_2018_17h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
c=power_weather_2019_17h_영암.reset_index().drop(['index','운형(운형약어)','풍향(16방위)'], axis=1).fillna(0)
power_weather_17h_영암_mean = (a+b+c)/3
power_weather_17h_영암_mean.corr()

#### 기상청의 기상데이터 - 발전소 센저정보의 기상데이터 - 발전량